In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
from sklearn.feature_extraction import text
import pickle
import re

In [2]:
yelp = pd.read_csv('./data/yelp.csv')
yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  10000 non-null  object
 1   date         10000 non-null  object
 2   review_id    10000 non-null  object
 3   stars        10000 non-null  int64 
 4   text         10000 non-null  object
 5   type         10000 non-null  object
 6   user_id      10000 non-null  object
 7   cool         10000 non-null  int64 
 8   useful       10000 non-null  int64 
 9   funny        10000 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 781.4+ KB


In [3]:
# 讀取csv檔僅保留"text"、"stars"兩個欄位
data = yelp[['text','stars']]
display(data.head(5))

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [4]:
# 將stars欄位內值大於等於4的轉成1，其餘轉成0
data.loc[data['stars']<4 , 'stars'] = 0 
data.loc[data['stars']>=4 , 'stars'] = 1 
data['text'] = data['text'].str.lower()


display(data)
# 1: positive, 0: negative

,text,stars
0,my wife took me here on my birthday for breakf...,1
1,i have no idea why some people give bad review...,1
2,love the gyro plate. rice is so good and i als...,1
3,"rosie, dakota, and i love chaparral dog park!!...",1
4,general manager scott petello is a good egg!!!...,1
...,...,...
9995,first visit...had lunch here today - used my g...,0
9996,should be called house of deliciousness!\n\ni ...,1
9997,i recently visited olive and ivy for business ...,1
9998,my nephew just moved to scottsdale recently so...,0


In [5]:
stop_list = ['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves',
        'he','him','his','himself','she','her','hers','herself,it','its','itself','they','them','their,theirs',
        'themselves','what','which','who','whom','this','that','these','those','am','is','are','was','were',
        'be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but',
        'if','or','because','as','until','while','of','at','by','for','with','about','against','between',
        'into','through','during','before','after','above','below','to','from','up','down','in','out','on',
        'off','over','under','again','further','then','once','here','there','when','where','why','how','all',
        'any','both','each','few','more','most','other','some','such','no','nor','not','only','own','same',
        'so','than','too','very','s','t','can','will','just','don','should','now','\n']

In [6]:
# with open('./data/stop_word.pickle', 'rb') as data:
#     delete_col = pickle.load(data)
len(text.ENGLISH_STOP_WORDS.union(stop_list))

329

len(set1)
stop_words = text.ENGLISH_STOP_WORDS.union(stop_list)
stop_words = text.ENGLISH_STOP_WORDS.union('My')
list(text.ENGLISH_STOP_WORDS)

In [7]:
# a_test = [['My','Lazy','Dog'],['My', 'Happy', 'Cat']]
# a_test = [['however my ,My Lazy Dog'],['My Happy Cat']]
data['text_split'] = ''
from gensim.parsing.preprocessing import remove_stopwords
count=0
for i in range(len(data)):
    # 去除停頓詞stop words
    data.text.iloc[i] = remove_stopwords(data.text.iloc[i])
    
    # 將text欄位內的文字利用分割符號切割
    split = re.split(';|,|\s|,\s|\.|\*|\n',data.iloc[i]['text'])
    split_new = [word for word in split if word.isalpha()]
    data.text_split.iloc[i] = split_new
data

,text,stars,text_split
0,wife took birthday breakfast excellent. weathe...,1,"[wife, took, birthday, breakfast, excellent, w..."
1,"idea people bad reviews place. goes you, every...",1,"[idea, people, bad, reviews, place, goes, you,..."
2,love gyro plate. rice good dig candy selection :),1,"[love, gyro, plate, rice, good, dig, candy, se..."
3,"rosie, dakota, love chaparral dog park!!! it's...",1,"[rosie, dakota, love, chaparral, dog, convenie..."
4,general manager scott petello good egg!!! deta...,1,"[general, manager, scott, petello, good, detai..."
...,...,...,...
9995,visit...had lunch today - groupon. ordered bru...,0,"[visit, had, lunch, today, groupon, ordered, b..."
9996,"called house deliciousness! item, item, blah b...",1,"[called, house, item, item, blah, blah, blah, ..."
9997,"recently visited olive ivy business week, 3 vi...",1,"[recently, visited, olive, ivy, business, week..."
9998,nephew moved scottsdale recently bunch friends...,0,"[nephew, moved, scottsdale, recently, bunch, f..."


In [8]:
set1 = {'ini'}
for i in range(len(data)):
    list1 = data.iloc[i]['text_split']
    for item in list1:
        set1.add(item)
set1.remove('ini')
len(set1)

26797

In [9]:
# words = [row[2] for row in data.itertuples(index=False, name=None)]
data['processed'] = data['text_split'].apply(lambda x: " ".join(x) )
data

,text,stars,text_split,processed
0,wife took birthday breakfast excellent. weathe...,1,"[wife, took, birthday, breakfast, excellent, w...",wife took birthday breakfast excellent weather...
1,"idea people bad reviews place. goes you, every...",1,"[idea, people, bad, reviews, place, goes, you,...",idea people bad reviews place goes you everyon...
2,love gyro plate. rice good dig candy selection :),1,"[love, gyro, plate, rice, good, dig, candy, se...",love gyro plate rice good dig candy selection
3,"rosie, dakota, love chaparral dog park!!! it's...",1,"[rosie, dakota, love, chaparral, dog, convenie...",rosie dakota love chaparral dog convenient sur...
4,general manager scott petello good egg!!! deta...,1,"[general, manager, scott, petello, good, detai...",general manager scott petello good detail let ...
...,...,...,...,...
9995,visit...had lunch today - groupon. ordered bru...,0,"[visit, had, lunch, today, groupon, ordered, b...",visit had lunch today groupon ordered bruschet...
9996,"called house deliciousness! item, item, blah b...",1,"[called, house, item, item, blah, blah, blah, ...",called house item item blah blah blah dont waz...
9997,"recently visited olive ivy business week, 3 vi...",1,"[recently, visited, olive, ivy, business, week...",recently visited olive ivy business week visit...
9998,nephew moved scottsdale recently bunch friends...,0,"[nephew, moved, scottsdale, recently, bunch, f...",nephew moved scottsdale recently bunch friends...


In [10]:
count = 0
for i in range(len(data)):
    aaa = data.text.iloc[i]
    bbb = data.processed.iloc[i]
    if aaa==bbb:
        count+=1
count

38

In [11]:
# 去除停頓詞stop words
stop_word = text.ENGLISH_STOP_WORDS.union(stop_list)

text_str = [row[3] for row in data.itertuples(index=False, name=None)]
# text = [row[2] for row in data.itertuples(index=False, name=None)]

# vec = CountVectorizer(stop_words='english' , analyzer=lambda x:x)
vec = CountVectorizer(stop_words=stop_word, analyzer='word', lowercase=False)
x = vec.fit_transform(text_str)

term = pd.DataFrame(x.toarray(), columns=vec.get_feature_names())
display(term)
vocab = vec.vocabulary_

,aa,aaa,aaaaaalright,aaaamazing,aaand,aah,aand,aaron,aarp,ab,...,zupa,zupas,zur,zuzu,zuzus,zweigel,zzzzzzzzzzzzzzzzz,éclairs,école,ém
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


AttributeError: 'CountVectorizer' object has no attribute 'vocabulary_S'

In [ ]:

transformer = TfidfTransformer(smooth_idf=True)
Z = transformer.fit_transform(x)
r = pd.DataFrame(Z.toarray(),columns=vec.get_feature_names())
display(r)
# r
# r.columns()

In [ ]:
r['wife'].sum()

In [ ]:
model = RandomForestClassifier(n_estimators=350, max_depth=25)


def k_fold(k, data, term):
    size = int(len(data)/k)
    acc = 0
    for i in range(0,k):
        # 訓練特徵         
        train_word = pd.concat([term[:i*size],term[(i+1)*size:]])
        # 訓練答案         
        train_star = pd.concat([data[:i*size],data[(i+1)*size:]])['stars']
        # 測試資料         
        test_word = term[i*size:(i+1)*size]
        # 測試答案        
        test_star = data[i*size:(i+1)*size]['stars']
        print('{}, {}'.format(len(train_word),len(test_word)))
        #訓練模型         
        model.fit(train_word, train_star)
        predict_ans = model.predict(test_word)
        acc += metrics.accuracy_score(test_star, predict_ans)
    return acc/k
        
        
print(len(data))
# del vec
# test_ans = [0,1]
k_fold(4,data,r)

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

words = list(data.text_split)
display(data.text_split)

In [ ]:
type(vec.get_feature_names())
# list(vocab)
# if 'aa' in list(vocab):
#     print('1')
# vocab['aaaaaalright']
vocab1 = dict(sorted(vocab.items(), key=lambda item: item[1]))

In [ ]:
w2v_model = Word2Vec(words, size=300,min_count=1)
# min_count要設定成1，因為默認5，會過濾掉出現較少次數的詞

word_embed1 = w2v_model[vocab1]

import numpy as np
# display(word_embed)
np.shape(word_embed1)

# w2v_model.wv.vocab
# len(list(vocab))
# np.shape(word_embed)

# term = 10000*26498
# word_vec = 26498*300(embed_size) 

# term排序跟不一樣
w2v_matrix = np.matmul(term, word_embed1) 
w2v_matrix
# np.shape(w2v_matrix)
w2v_result = pd.DataFrame(w2v_matrix)
w2v_result

In [ ]:
del model
model = RandomForestClassifier(n_estimators=350, max_depth=25)


def k_fold(k, data, term):
    size = int(len(data)/k)
    acc = 0
    for i in range(0,k):
        # 訓練特徵         
        train_word = pd.concat([term[:i*size],term[(i+1)*size:]])
        # 訓練答案         
        train_star = pd.concat([data[:i*size],data[(i+1)*size:]])['stars']
        # 測試資料         
        test_word = term[i*size:(i+1)*size]
        # 測試答案        
        test_star = data[i*size:(i+1)*size]['stars']
        print('{}, {}'.format(len(train_word),len(test_word)))
        #訓練模型         
        model.fit(train_word, train_star)
        predict_ans = model.predict(test_word)
        acc += metrics.accuracy_score(test_star, predict_ans)
        print(metrics.accuracy_score(test_star, predict_ans))
    return acc/k
        
        
print(len(data))
# del vec
k_fold(4,data,w2v_result)